Team Members-

1) Abhishek Agrawal

2) Ankur Singh

3) Atharva Dixit

4) Ruthvik Kamarasu

# 274 Business Intelligence Platform

### Setup

In [1]:
!pip install -qU gradio openai llama-index llama-index-embeddings-huggingface llama-index-llms-openrouter

### Imports

In [2]:
from pathlib import Path
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from llama_index.core.embeddings import resolve_embed_model

### Initializations

In [3]:
BASE_URL = "https://openrouter.ai/api/v1"

#API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [4]:
PERSIST_DIR = Path("./storage")
DATA_DIR = Path("./data")
assert DATA_DIR.exists(), "Please create the data directory and upload all your files!"

### Setting Embedding and LLM model

In [5]:
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
Settings.llm = OpenRouter(api_key=API_KEY, model="mistralai/mistral-7b-instruct:free")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Indexing Documents

In [6]:
if not PERSIST_DIR.exists():
    documents = SimpleDirectoryReader(DATA_DIR).load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()

In [7]:
response = query_engine.query("List major Mosaic Influencers in 2023 (Mosaic 10-Q)")
print(response)

 Based on the provided context information, there is no explicit mention of major Mosaic influencers in 2023. However, the table of contents mentions the financial information for the three and nine months ended September 30, 2023, which could be used to identify key financial metrics and performance indicators for the company during that time period. Additionally, the table of contents also mentions the signatures of the registrant, which could potentially provide insight into the leadership and decision-making processes of the company.


## Dashboard

In [8]:
import pandas as pd
import gradio as gr

In [9]:
df = pd.read_csv(DATA_DIR/"WPU0652026A.csv")
df.DATE = pd.to_datetime(df.DATE)

In [10]:
def respond(message, chat_history):
    bot_message = str(query_engine.query(message))
    chat_history.append((message, bot_message))
    return "", chat_history

In [11]:
APP_TITLE_MD = "# CMPE-274 Business Dashboard"
APP_INTRO_MD = """Explore the dynamic interplay of urbanization and agrifood systems transformation across the rural–urban continuum. This dashboard provides insights into global and local food security challenges, innovations in sustainable agriculture, and the impact of economic, environmental, and social factors on nutrition and health. Dive into data-driven analyses and visualizations that highlight the state of food security and the pathways to achieving healthier diets for all.
"""

RAG_INTRO_MD = """Explore the innovative applications of generative AI in the realm of food security. This section explores how AI technologies can forecast agricultural outputs, optimize food distribution, and enhance nutritional planning to mitigate hunger. It discusses the potential of AI to transform food systems through predictive analytics, machine learning models, and real-time data processing, fostering more resilient and efficient food production and supply chains.
"""

PLOT_INTRO_MD = """Our interactive plots offer a visual narrative of food security data, illustrating trends and patterns in hunger, malnutrition, and dietary shifts over time. By visualizing the data from various angles, users can grasp the complexities of food supply chains, the impacts of urbanization on food availability, and the critical role of technological and policy innovations in shaping food systems.
"""

REPORT_INTRO_MD = """Dive into detailed reports that synthesize the latest findings on food security, agrifood systems, and nutritional trends. Each report combines rigorous analysis with policy recommendations, providing stakeholders with the information needed to design interventions that enhance food security and promote equitable access to nutritious diets.
"""

EXEC_REPORT_HTML = """
<table border="1">
    <tr>
        <th></th>
        <th>Mosaic</th>
        <th>Food Security</th>
    </tr>
    <tr>
        <td>Food Security Measures</td>
        <td>One approach to improving food security is through the development of urban food systems. This involves creating a diverse and complementary range of food production, distribution, and consumption systems that are tailored to the needs of urban populations.</td>
        <td>Strengthening agricultural productivity and resilience: This involves investing in research and development, improving access to inputs such as seeds and fertilizers, and promoting sustainable agricultural practices.</td>
    </tr>
    <tr>
        <td>Climate Change</td>
        <td>Overall, the mosaic document suggests that climate change is a complex and multifaceted issue that requires a comprehensive and coordinated response from governments, civil society</td>
        <td>The SOFI document discusses the risks of extreme events and disasters in the context of climate change. It highlights the potential impacts of climate change on agriculture, food security, and nutrition. The document also discusses the importance of adapting to these risks and the role of sustainable rural livelihoods in this process.</td>
    </tr>
    <tr>
        <td>Wars and Political Uncertainties</td>
        <td>The mosaic document discusses contingencies and their impact on various countries, including Georgia, Iraq, Israel, Jordan, Kuwait, Lebanon, Oman, Palestine, Qatar, Saudi Arabia, Syria, Turkey, and the United Arab Emirates, as well as political uncertainties in these regions.</td>
        <td>The SOFI document provides information on various political uncertainties and conflicts in countries such as Cambodia, Indonesia, Lao People's Democratic Republic, Malaysia, Myanmar, Philippines, Singapore, Türkiye, United Arab Emirates, and Yemen, which may impact the stability and security of the region.</td>
    </tr>
</table>

"""

In [ ]:
with gr.Blocks("📊 Business Intelligence Platform") as demo:
    gr.Markdown(APP_TITLE_MD)
    gr.Markdown(APP_INTRO_MD)

    with gr.Tabs() as tabs:

        # RAG tab
        with gr.TabItem("RAG Chatbot", id=0):
            gr.Markdown(RAG_INTRO_MD)
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            msg.submit(respond, [msg, chatbot], [msg, chatbot])


        # PLOT tab
        with gr.TabItem("Plot", id=1):
            gr.Markdown(PLOT_INTRO_MD)
            gr.LinePlot(df, "DATE", "WPU0652026A", x_title="Date",
                        interactive=True, width=800, height=400,
                        show_actions_button=True, show_label=False)


        # Report tab
        with gr.TabItem("Executive Report", id=2):
            gr.Markdown(REPORT_INTRO_MD)
            gr.HTML(EXEC_REPORT_HTML)


if __name__ == "__main__":
    demo.launch(debug=True)



/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:626: UserWarning: Cannot load 📊 Business Intelligence Platform. Caught Exception: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: '📊 Business Intelligence Platform'.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20a61e3dae3228d0d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
